In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 29 14:12:54 2023

@author: Alberto
"""

import numpy as np
import rasterio
import math
from shapely.geometry import Point
import geopandas as gpd
import cv2
import os
import rasterio.mask
import sys
import pandas as pd
sys.path.append(r'E:/gitlab/geoloc2/Detecciondeterrenos')
###path de yolo dentro de computadora
os.chdir(r'C:/Users/ASUS/Inteligencia_Artificial/yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
from affine import Affine
from shapely.ops import cascaded_union

raster_mun=r"E:\rastermunicpial\nau.tif"
shape_o=gpd.read_file(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\geoshapes\Full_manzanas\Manzana_Naucalpan.shp")
shape_o=shape_o.to_crs('3857')
shape=cascaded_union(shape_o.buffer(50))
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster_mun,metros=500)
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy
def suma_interior_contorno(matriz,c):
    img = matriz#cv2.imread(r"C:\Users\ASUS\Desktop\atlacoa1.tif")
    pts = np.array([x[0] for x in c])#np.array([[10,150],[150,100],[300,150],[350,100],[310,20],[35,10]])
    rect = cv2.boundingRect(pts)
    x,y,w,h = rect
    croped = img[y:y+h, x:x+w].copy()
    pts = pts - pts.min(axis=0)
    mask = np.zeros(croped.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
    dst = cv2.bitwise_and(croped, croped, mask=mask)
    return np.mean(dst[dst!=0],dtype="int")
generar=0
count=0
#124
with rasterio.open(raster_mun) as scr:
    with tqdm.tqdm(total=alto*ancho) as pbar:
        for j in range(ancho):#ancho
            for i in range(alto):#alto
                generar=0
                # if count>=446: 
                #     generar=0
                # else:
                #     count+=1
                #     pbar.update(1)
                #     continue
                label=raster_mun.replace('\\','/').split('/')[-1][:-4]+'_'
                nameimg=label.lower()+str(i)+'_'+str(j)
                cuadro=[]
                for k in range(2):
                    for l in range(2):
                        cuadro.append((minx+(maxx-minx)/ancho*(j+k),maxy-(maxy-miny)/alto*(i+l)))
                cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                cvees=[]
                for punto in cuadro:
                    x=float(punto[0])
                    y=float(punto[1])
#                     mini_df=
                    if shape.contains(Point(x,y)):
                        generar=1
#                     else:
#                         pbar.update(1)
                if generar==1:
                    shapes=[{'type':'Polygon','coordinates':[cuadro]}]
                    manzana,out_transform = rasterio.mask.mask(scr, shapes, crop=True)
#                     out_meta = scr.meta
#                     out_meta.update({'driver': 'GTiff',
#                     'height': manzana.shape[1],
#                     'width': manzana.shape[2],
#                     'transform': out_transform})
#                     raster = r'D:/neza/ma1a.tif'
#                     with rasterio.open(raster, 'w', **out_meta) as dest:
#                         dest.write(manzana)   
#                     with rasterio.open(raster) as src:
                    f=out_transform
                    sa=gpd.GeoSeries(Point(f[2],f[5]),crs="3857")
                    saa=sa.to_crs("6364")
                    angulo=float(str(saa[0]).split()[-1].replace(")",""))
                    metrosxpixel=f[0]
                    crs=scr.meta.get("crs")
                    manzana=np.moveaxis(manzana,0,-1)
                    manzana=cv2.cvtColor(manzana,cv2.COLOR_RGB2BGR)
                    img=manzana#cv2.imread(raster)
                    edges = cv2.bitwise_not(cv2.Canny(img[:,:,0],100,255))/255

                    # cv2_imshow("s",np.array(np.stack([edges,edges,edges],axis=-1)*img,dtype="uint8"))
                    rgb_edges=np.stack([edges,edges,edges],axis=-1)
                    rse=np.array(rgb_edges*img,dtype="uint8")
                    X = img.reshape((-1,3))
                    f=1
                    axis_n1=range(3)
                    X=np.array(X/f,dtype="uint8")*f
                    Z = np.float32(X)
                    kmeans=pd.DataFrame(Z,columns=axis_n1)
                    kmeans["grupo"]=0
                    centros=np.array([[40,40,40],[132,117,126],[57,72,74],[124,152,178],[106,106,166],[16,15,205],[189, 194, 197]])#[parametro1,parametro2,parametro3]
                    kmeans["menor"]=1e15
                    for i,pt in enumerate(centros):
                        kmeans["d"+str(i)]=list(np.sqrt(np.sum((centros[i]-Z)**2,axis=1)))
                        kmeans.loc[kmeans["d"+str(i)]<=kmeans["menor"],"grupo"]=i
                        kmeans.loc[kmeans["d"+str(i)]<=kmeans["menor"],"menor"]=kmeans["d"+str(i)][kmeans["d"+str(i)]<=kmeans["menor"]]
                    res=centros[kmeans["grupo"].values]
                    res2 = np.array(res.reshape((img.shape[0],img.shape[1],3)),dtype="uint8")
#                         cv2.namedWindow("result", cv2.WINDOW_NORMAL)
                    # cv2_imshow("result",res2)
                    img=rse.copy()
                    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
                    hsv_low=np.array([ 0,  0, 73], dtype="uint8")
                    hsv_high= np.array([179, 255, 255], dtype="uint8")
                    mask = cv2.inRange(hsv, hsv_low, hsv_high)
                    _,ix=cv2.threshold(mask,70,255,1)
                    ix=ix/255
                    black=np.full((ix.shape[0],ix.shape[1]),0)
                    for x in range(ix.shape[1]):
                        s=ix[:,x]
                        for pases in range(10):
                            for i in range(len(s)-1):
                                if s[i]==0 and s[i-1]==1 and s[i+1]==1:
                                    s[i]=1
                        contador=0
                        altura=0
                        alt=[]
                        for i in range(len(s)):
                            if s[i]!=0 and s[i]!=1:
                                print(s[i])
                            if i==0 and s[0]==0:
                                contador=0
                                altura=0
                            elif i==0 and s[0]==1:
                                contador=1
                                altura=0
                            else:
                                if s[i]==s[i-1]:
                                    if s[i-1]==0:
                                        contador=contador
                                        altura=altura
                                    else:
                                        contador+=1
                                        altura=altura
                                else:
                                    if s[i-1]==0:
                                        if contador<np.round(5*metrosxpixel,0):
                                            contador=contador
                                            altura=altura
                                        else:
                                            contador=1
                                            altura=0
                                    else:
                                        contador=contador
                                        if contador<np.round(5*metrosxpixel,0):
                    #                         print(str(x)+" "+str(i))
                                            altura=altura+(contador)*math.cos(angulo*math.pi/180)*metrosxpixel*3
                    #                     elif contador>10:
                    #                         altura=altura
                                        else:
                                            altura=(contador)*math.cos(angulo*math.pi/180)*metrosxpixel*3
                            alt.append(altura)
                    #         black[x,i]=altura
                        black[:,x]=alt
                    white=np.full((img.shape[0],img.shape[1]),0)
                    # white1=np.full((img.shape[0],img.shape[1]),0)
                    white[[res2==[57,72,74]][0][:,:,0]]=255
                    white[[res2==[106,106,166]][0][:,:,0]]=255
                    white[[res2==[189, 194, 197]][0][:,:,0]]=255
                    white[[res2==[16,15,205]][0][:,:,0]]=255
                    f=np.array(np.where([res2[:,:,2]>=166 , res2[:,:,0]!=188])) 
                    y1=[i for i,y in enumerate(f[0,:]) if y==1]
                    white=white*rgb_edges[:,:,0]
                    contours,hist=cv2.findContours(np.array(white,dtype="uint8"),0, cv2.CHAIN_APPROX_SIMPLE)

#                     cv2.drawContours(white,contours,-1,(700),-1)
                    area=[cv2.contourArea(x) for x in contours] 
                    for cnt in contours:
                        x,y=cnt.mean(axis=0,dtype="int")[0]
                        cv2.fillPoly(white, cnt, int(black[y,x]))

                    f=f[1:,y1]
                    f=np.moveaxis(f,0,-1)
                    a=np.zeros_like(img[:,:,0])
                    a[[res2==centros[3]][0][:,:,0]]=.5
                    a[[res2==centros[4]][0][:,:,0]]=1
                    a[[res2==centros[-1]][0][:,:,0]]=1
                    array=black.copy()
                    mean = 1.25*np.mean(black[black>0])
                    array[array > mean]= mean
                    matriz=array*rgb_edges[:,:,0]*white/255.

                    contour,histe=cv2.findContours(a, 2,1)
                    m=np.zeros_like(matriz)
                    for cnt in contour:
                        rect = cv2.minAreaRect(cnt)
                        box = cv2.boxPoints(rect)
                        box = np.int0(box)
                        altura=max(0,int(suma_interior_contorno(matriz,cnt)))
                        cv2.drawContours(m,[cnt],0,(altura,altura,altura),-1)

                    # crs=3857
                    matriz=m
                    h,w=matriz.shape
                    transform=out_transform#src.meta.get("transform")
                    out_meta=({"driver": "GTiff",
                                "height": matriz.shape[0],
                                "width": matriz.shape[1],
                                "count":1,
                                "dtype":np.float32,
                                "crs":crs,
                              'transform':transform})
#                         if j==0 and i==0:
                    with rasterio.open(r"E:\rastermunicpial\naucalpan/fnuevas500_"+nameimg+".tif",'w', **out_meta) as dest_file:
                        dest_file.write(matriz, 1)
#                         else:
#                             with rasterio.open(r"E:\geoshapes/"+nameimg+".tif",'', **out_meta) as dest_file:
#                                 dest_file.write(matriz, 1)
                    pbar.update(1)
                else:
                    pbar.update(1)
##106

In [ ]:
from pathlib import Path
import os
from osgeo import gdal

folder_path = r"C:\Users\ASUS\salida\lerma"
BASE_PATH = os.path.dirname(os.path.abspath(folder_path))
folder = Path(folder_path)

l = []

for f in folder.glob('f*.tif'):
    f_path = f.as_posix()
    l.append(f_path)

vrt_path = os.path.join(BASE_PATH, 'prov1_vrt.vrt')
vrt = gdal.BuildVRT(vrt_path, l)

result = os.path.join(BASE_PATH, 'mer1ged.tif')
gdal.Translate(result, vrt, format='GTiff')